The process of making shuffled motifs can be made in the same time as the normal motifs but since I already did the prior instead of repeating the process i make code just to the shuffled

In [1]:
import pickle
import random
import pandas
import pickle
import itertools
import re
from pg import DB
db = DB()

In [4]:
n=8
fileName = "files/intermediates/motifs_to_proteins_shuffled_4_to_" + str(n) + ".pkl"
motifs_to_proteins_shuffled = pickle.load( open( fileName, "rb" ) )
distanceMatrix_df = pandas.read_pickle(open( "files/intermediates/distanceMatrix.pkl", "rb" ))

In [5]:
len(motifs_to_proteins_shuffled)

65536

In [20]:
len(motifs_to_proteins_shuffled)
# motifs_to_proteins_series = pandas.Series(motifs_to_proteins)
lengths = []
for motif in motifs_to_proteins_shuffled:
    if( len(motifs_to_proteins_shuffled[motif]) >4):
        lengths.append(len(motifs_to_proteins_shuffled[motif]))
lengths_series = pandas.Series(lengths)
lengths_series.describe()

count    65536.000000
mean        84.278366
std         59.906043
min         16.000000
25%         51.000000
50%         66.000000
75%         95.000000
max       1227.000000
dtype: float64

In [23]:
db.query("ALTER TABLE motifs ADD COLUMN shuffled_proteins VARCHAR[];")

In [31]:
%%time
%%capture
for motif in motifs_to_proteins_shuffled:
    if(db.query("SELECT motif_string FROM motifs where motif_string = '" + motif + "' and shuffled_subgraph NOTNULL").ntuples() > 0):
        continue
    if( len(motifs_to_proteins_shuffled[motif]) < 2):
        continue
    shuffled_proteins = motifs_to_proteins_shuffled[motif]
    motif_to_shuffled_subgraphs = [];
    for protein_i in shuffled_proteins:
        row = []
        for protein_j in shuffled_proteins:
            if(protein_i != protein_j):
                  row.append(distanceMatrix_df[protein_i][protein_j])
        motif_to_shuffled_subgraphs.append(row)
    do_not_print = db.query("UPDATE motifs SET shuffled_proteins = '" + str(shuffled_proteins) + "' , shuffled_proteins_length = '" + str(len(shuffled_proteins)) + "', shuffled_subgraph = '" + str(motif_to_shuffled_subgraphs) +  "' WHERE motif_string = '" + motif + "';")
    

ProgrammingError: ERROR:  syntax error at or near "QRSL1"
LINE 1: UPDATE motifs SET shuffled_proteins = '['QRSL1', 'GTPBP10', ...
                                                 ^


CPU times: user 10.4 s, sys: 150 ms, total: 10.6 s
Wall time: 10.7 s
